In [ ]:
def dfs(at):
    # print('at', at)
    if visited[at]: return 
    visited[at] = True
    neighbors = adj[at]
    for next in neighbors:
        # print(next)
        dfs(next)

dfs(x)

In [177]:
def acyclic(adj):
    n = len(adj)
    visited = [False] * n
    recStack = [False] * n

    def dfs(v):
        visited[v] = True
        recStack[v] = True

        neighbors = adj[v]
        for next in neighbors:
            if not visited[next]:
                if dfs(next):
                    return True
            elif recStack[next]:
                return True
        recStack[v] = False
        return False

    for v in range(n):
        if dfs(v):
            return 0
    
    return 1


In [177]:
def acyclic(adj):
    n = len(adj)
    visited = [False] * n
    recStack = [False] * n

    def dfs(v):
        visited[v] = True
        recStack[v] = True

        neighbors = adj[v]
        for next in neighbors:
            if not visited[next]:
                if dfs(next):
                    return True
            elif recStack[next]:
                return True
        recStack[v] = False
        return False

    for v in range(n):
        if dfs(v):
            return 0
    
    return 1


In [178]:
# adj = [[1], [2], []]  # 0 -> 1 -> 2
# adj = [[1], [2], [3, 4], [5], [1], [0]]  # multiple cycles involving all nodes
# adj = [[1], [], [3], []]  # 0 -> 1 and 2 -> 3
adj = [[1], [], [3], []]  # 0 -> 1 and 2 -> 3
acyclic(adj) 

1

In [179]:
def test_acyclic():
    # Test case 1: Simple acyclic graph (tree structure)
    adj = [[1], [2], []]  # 0 -> 1 -> 2
    assert acyclic(adj) == 1, "Test case 1 failed: Expected 1, found 0"

    # Test case 2: Simple cyclic graph (single cycle)
    adj = [[1], [2], [0]]  # 0 -> 1 -> 2 -> 0
    assert acyclic(adj) == 0, "Test case 2 failed: Expected 0, found 1"

    # Test case 3: Graph with a self-loop
    adj = [[1], [2], [2]]  # 2 -> 2 (self-loop)
    assert acyclic(adj) == 0, "Test case 3 failed: Expected 0, found 1"

    # Test case 4: Disconnected acyclic graph
    adj = [[1], [], [3], []]  # 0 -> 1 and 2 -> 3
    assert acyclic(adj) == 1, "Test case 4 failed: Expected 1, found 0"

    # Test case 5: Disconnected graph with a cycle
    adj = [[1], [0], [3], [4], [2]]  # 0 -> 1 -> 0 and 2 -> 3 -> 4 -> 2
    assert acyclic(adj) == 0, "Test case 5 failed: Expected 0, found 1"

    # Test case 6: Large acyclic graph
    adj = [[1, 2], [3], [4], [], []]  # 0 -> 1 -> 3 and 0 -> 2 -> 4
    assert acyclic(adj) == 1, "Test case 6 failed: Expected 1, found 0"

    # Test case 7: Large graph with multiple cycles
    adj = [[1], [2], [3, 4], [5], [1], [0]]  # multiple cycles involving all nodes
    assert acyclic(adj) == 0, "Test case 7 failed: Expected 0, found 1"

    print("All test cases passed.")

test_acyclic()


All test cases passed.


## Topological Sort

In [218]:
def dfs(adj, used, order, x):
    used[x] = True
    for next in adj[x]:
        if not used[next]:
            dfs(adj, used, order, next)
    order.append(x)

def toposort(adj):
    n = len(adj)
    used = [False] * n
    order = []
    for v in range(n):
        if not used[v]:
            dfs(adj, used, order, v)
    order.reverse()
    return order

In [219]:
adj = [[1], [2], [3], [4], []]
toposort(adj)

[0, 1, 2, 3, 4]

In [220]:
def test_toposort():
    # Test case 1: Simple DAG
    adj = [[1, 2], [3], [3], []]
    expected_output = [0, 2, 1, 3]  # or [0, 1, 2, 3]
    assert toposort(adj) in [expected_output, expected_output[::-1]], \
        "Test case 1 failed"

    # Test case 2: Another simple DAG
    adj = [[], [0], [1], [2]]
    expected_output = [3, 2, 1, 0]  # or any other valid topological order
    assert toposort(adj) in [expected_output, expected_output[::-1]], \
        "Test case 2 failed"

    # Test case 3: Single node graph
    adj = [[]]
    expected_output = [0]
    assert toposort(adj) == expected_output, "Test case 3 failed"

    # Test case 4: Two disconnected nodes
    adj = [[], []]
    expected_output = [0, 1]  # or [1, 0]
    assert toposort(adj) in [expected_output, expected_output[::-1]], \
        "Test case 4 failed"

    # Test case 5: Graph with a self-loop (not a DAG)
    adj = [[0], [2], [1]]
    # Topological sort should not be possible for a graph with cycles
    # The function could return an empty list or partial ordering in such cases
    assert toposort(adj) == [], "Test case 5 failed: Expected empty result due to cycle"

    # Test case 6: More complex DAG
    adj = [[1, 2], [2], [3], []]
    expected_output = [0, 1, 2, 3]
    assert toposort(adj) == expected_output, "Test case 6 failed"

    # Test case 7: DAG with multiple valid topological orders
    adj = [[1], [2], [3], [4], []]
    expected_output = [0, 1, 2, 3, 4]  # or any other valid topological order
    assert toposort(adj) == expected_output, "Test case 7 failed"

    print("All test cases passed.")

test_toposort()


AssertionError: Test case 5 failed: Expected empty result due to cycle

## strongly connected components

In [243]:
set([1,2,2])

{1, 2}

In [236]:

def number_of_strongly_connected_components(adj):
    n = len(adj)
    visited = [False] * n
    post_order = []

    # Helper function for DFS
    def dfs(v, graph, collect_post_order=False):
        visited[v] = True
        for neighbor in graph[v]:
            if not visited[neighbor]:
                dfs(neighbor, graph, collect_post_order)
        if collect_post_order:
            post_order.append(v)

    # Reverse the graph
    def reverse_graph(graph):
        rev_graph = [[] for _ in range(len(graph))]
        for v in range(len(graph)):
            for neighbor in graph[v]:
                rev_graph[neighbor].append(v)
        return rev_graph

    # Step 1: DFS on the reversed graph to get post order
    rev_adj = reverse_graph(adj)
    for i in range(n):
        if not visited[i]:
            dfs(i, rev_adj, collect_post_order=True)

    # Reset visited for second pass
    visited = [False] * n
    sccs = []
    # print(post_order)
    while len(post_order)>0:
        v = post_order.pop()
        if not visited[v]:
            component = []
            tmp = [False] * n
            dfs(v, adj, collect_post_order=False)
            component = [i for i in range(n) if visited[i]]
            sccs.append(component)
            for node in component:
                visited[node] = False  
    return sccs

In [246]:
def number_of_strongly_connected_components(adj):
    n = len(adj)
    visited = [False] * n
    post_order = []

    # Helper function for DFS
    def dfs(v, graph, collect_post_order=False):
        visited[v] = True
        for neighbor in graph[v]:
            if not visited[neighbor]:
                dfs(neighbor, graph, collect_post_order)
        if collect_post_order:
            post_order.append(v)

    # Reverse the graph
    def reverse_graph(graph):
        rev_graph = [[] for _ in range(len(graph))]
        for v in range(len(graph)):
            for neighbor in graph[v]:
                rev_graph[neighbor].append(v)
        return rev_graph

    # Step 1: DFS on the reversed graph to get post order
    rev_adj = reverse_graph(adj)
    for i in range(n):
        if not visited[i]:
            dfs(i, rev_adj, collect_post_order=True)

    # Reset visited for second pass
    visited = [False] * n
    sccs = []

    # Step 2: DFS on the original graph in decreasing order of post numbers
    while post_order:
        v = post_order.pop()
        if not visited[v]:
            component = []
            dfs(v, adj)
            component = [i for i in range(n) if visited[i]]
            if component not in sccs:
                sccs.append(component)
            for node in component:
                visited[node] = False  # Unmark nodes for future components

    return len(sccs)



In [247]:
adj = [[1], [2, 3], [0, 4], [4], [3, 5], [6], [4]]
# SCCs: {0, 1, 2}, {3, 4, 5, 6}
number_of_strongly_connected_components(adj)

2

In [248]:
def test_number_of_strongly_connected_components():
    # Test case 1: Single SCC
    adj = [[1], [2], [0]]  # One cycle, all nodes are mutually reachable
    assert number_of_strongly_connected_components(adj) == 1, "Test case 1 failed"

    # Test case 2: Two separate SCCs
    adj = [[1], [0], [3], [2]]  # Two cycles: 0 <-> 1 and 2 <-> 3
    assert number_of_strongly_connected_components(adj) == 2, "Test case 2 failed"

    # Test case 3: Linear chain, each node its own SCC
    adj = [[1], [2], [3], []]  # Each node can only reach itself
    assert number_of_strongly_connected_components(adj) == 4, "Test case 3 failed"

    # Test case 4: Multiple SCCs with varying sizes
    adj = [[1, 2], [0], [1], [4], [5], [3]]  # Three SCCs: {0, 1, 2}, {3, 4, 5}
    assert number_of_strongly_connected_components(adj) == 3, "Test case 4 failed"

    # Test case 5: Single node, self-loop
    adj = [[0]]  # Single node with a self-loop
    assert number_of_strongly_connected_components(adj) == 1, "Test case 5 failed"

    # Test case 6: Disconnected nodes, each node its own SCC
    adj = [[], [], []]  # No edges, three separate SCCs
    assert number_of_strongly_connected_components(adj) == 3, "Test case 6 failed"

    # Test case 7: Complex graph with nested SCCs
    adj = [[1], [2, 3], [0, 4], [4], [3, 5], [6], [4]]
    # SCCs: {0, 1, 2}, {3, 4, 5, 6}
    assert number_of_strongly_connected_components(adj) == 2, "Test case 7 failed"

    print("All test cases passed.")

test_number_of_strongly_connected_components()


AssertionError: Test case 4 failed